# Real Estate Data Analysis


* [1. Summary of Business Task](#summary_1)
* [2. Gathering Data](#gathering_data)
* [3. Documentation of cleaning of data](#cleaning_3)

#  1. Summary of Business Task<a class="anchor" id="summary_1"></a>

Collect data of Real Estate in Middlesex County and use it to predict house prices based on features.

#  2. Gathering Data <a class="anchor" id="gathering_data"></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import numpy as np

url = 'https://www.redfin.com/county/1902/NJ/Middlesex-County'
html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'lxml')

In [2]:
response = requests.get(url)

html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'lxml')

cities = [
    'https://www.redfin.com/city/35780/NJ/Edison',
    'https://www.redfin.com/city/2768/NJ/Carteret',
    'https://www.redfin.com/city/22091/NJ/Cranbury',
    'https://www.redfin.com/city/4785/NJ/Dunellen',
    'https://www.redfin.com/minorcivildivision/743/NJ/East-Brunswick-Township',
    'https://www.redfin.com/city/7886/NJ/Helmetta',
    'https://www.redfin.com/city/8051/NJ/Highland-Park',
    'https://www.redfin.com/city/8884/NJ/Jamesburg',
    'https://www.redfin.com/city/11649/NJ/Metuchen',
    'https://www.redfin.com/city/11707/NJ/Middlesex',
    'https://www.redfin.com/city/11901/NJ/Milltown',
    'https://www.redfin.com/minorcivildivision/746/NJ/Monroe-Township',
    'https://www.redfin.com/city/13201/NJ/New-Brunswick',
    'https://www.redfin.com/minorcivildivision/742/NJ/North-Brunswick-Township',
    'https://www.redfin.com/city/24831/NJ/Old-Bridge',
    'https://www.redfin.com/city/15081/NJ/Perth-Amboy',
    'https://www.redfin.com/minorcivildivision/744/NJ/Piscataway-Township',
    'https://www.redfin.com/minorcivildivision/748/NJ/Plainsboro-Township',
    'https://www.redfin.com/city/16859/NJ/Sayreville',
    'https://www.redfin.com/city/17563/NJ/South-Amboy',
    'https://www.redfin.com/minorcivildivision/750/NJ/South-Brunswick-Township',
    'https://www.redfin.com/city/17747/NJ/South-Plainfield',
    'https://www.redfin.com/city/17755/NJ/South-River',
    'https://www.redfin.com/city/17842/NJ/Spotswood',
    'https://www.redfin.com/city/26641/NJ/Woodbridge'
]

pages = cities.copy()

for city in cities:
    html_text = requests.get(city).text
    soup = BeautifulSoup(html_text, 'lxml')
    links = soup.find_all('a', class_='clickable goToPage')
    for link in links:
        pages.append("https://www.redfin.com" + link['href'])  # Construct the full URL for each page


print(len(pages))

38


In [3]:
properties = []
for page in pages:
    current_page = requests.get(page).text
    soup = BeautifulSoup(current_page, 'lxml')
    props = soup.find_all('div', class_='PhotoSlider photoContainer')
    for property in props:
        link = property.find('a')
        properties.append("https://www.redfin.com" + link['href'])
print(len(properties))

1026


In [4]:
properties = list(set(properties))
print(len(properties))

925


#### Details to scrape

- Address
- Price
- Beds
- Baths
- Sq ft
- Basement
- Rooms
- Garage
- Property Type
- Year Built
- Price/Sq.Ft. (calculate later)
- Annual Tax
- walk score
- bike score

- https://www.walkscore.com/score/1232-dakota-rd-n-brunswick-township-nj-08902


In [ ]:
df = pd.DataFrame(columns=['Price', 'Beds', 'Baths', 'Sq-ft', 'Basement', 'Garage', 'Rooms', 'Annual Tax', 'Property Type', 'Style', 'Year Built', 'Walkable Score', 'Bikeable Score', 'Address'])

In [ ]:
for property in properties[562:]:
    print(property)
    property_url = property
    current_page = requests.get(property_url).text
    soup = BeautifulSoup(current_page, 'lxml')
    address = soup.find('header', class_='address').text
    stats = soup.find_all('div', class_='statsValue')
    price = stats[0].text
    beds = stats[1].text
    baths = stats[2].text
    sqft = soup.find('div', class_='stat-block sqft-section').text
    basement = None
    garage = 0
    rooms = None
    annual_tax = None
    property_type = ''
    style = ''
    yr_built = None
    walk_score = None
    bike_score = None

    text = soup.find_all('div', class_='amenity-group')
    for para in text:
        if 'Has Garage' in para.text or 'Has Attached Garage' in para.text:
            garage = 1
        if '# of Rooms:' in para.text:
            rooms = para.text.split(':')[-1]
        if 'Tax Annual Amount' in para.text:
            annual_tax = para.text.split(':')[-1]
        if 'Basement Information' in para.text:
            basement = para.text.split(':')[-1]
    text = soup.find_all('div', class_='DPTableDisplay')
    for para in text:
        if 'Built' in para.text:
            yr_built = para.text.split('Built')[1].split(' ')[0]
        if 'Style' in para.text:
            style = para.text.split('Style')[1].split('Year')[0]

    data = soup.find_all('div', class_='percentage')
    if data:
        walk_score = data[0].text
        bike_score = data[-1].text
    row_data = [price, beds, baths, sqft, basement, garage, rooms, annual_tax, property_type, style, yr_built, walk_score, bike_score, address]
    length = len(df)
    df.loc[length] = row_data

In [44]:
df.to_csv('real_estate.csv', index = False)

#  3. Documentation of Cleaning of Data<a class="anchor" id="cleaning_3"></a>

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [72]:
df = pd.read_csv('real_estate.csv')
df

,Price,Beds,Baths,Sq-ft,Basement,Garage,Rooms,Annual Tax,Property Type,Style,Year Built,Walkable Score,Bikeable Score,Address
0,"$665,000",4,2.5,"2,418Sq Ft",YesFull,1,NaN,"$11,409",NaN,Single Family Residential,1979Year,44/100,40/100,"43 Sand Hills Rd,Kendall Park, NJ 08824"
1,"$249,900",2,1,936Sq Ft,0,0,4,"$5,189",NaN,Condo/Co-op,1977Year,28/100,41/100,"280 College Dr #280,Edison, NJ 08817"
2,"$399,000",3,1,"1,008Sq Ft","Full, Exterior Entry, Storage Space, Laundry ...",0,5,"$4,926",NaN,Single Family Residential,1963Year,30/100,32/100,"616 Plainfield Ave,Piscataway, NJ 08854"
3,"$599,900",3,3.5,—Sq Ft,"Finished, Bath Full, Laundry Facilities",1,6,"$4,385",NaN,Single Family Residential,1954Year,19/100,32/100,"313 Wadsworth Ave,South Plainfield, NJ 07080"
4,"$289,000",2,1,—Sq Ft,Crawl Only,0,5,"$4,154",NaN,Single Family Residential,1923Year,41/100,35/100,"214 Seaview Ave,Old Bridge, NJ 08879"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,"$459,900",4,2,"1,468Sq Ft","Full, Laundry Facilities",1,9,"$4,220",NaN,Single Family Residential,1926Year,81/100,67/100,"12 Holmes Ave,South River, NJ 08882"
932,"$894,990+",4,2.5,"2,999Sq Ft",NaN,0,NaN,NaN,NaN,NaN,NaN,0/100,25/100,"Brookdale Plan,Old Bridge, NJ 08857"
933,"$650,000",4,3.5,"2,672Sq Ft","Full, Finished, Bath Half, Den, Interior Entr...",1,11,"$12,497",NaN,Single Family Residential,1961Year,57/100,55/100,"541 Allgair Ave,North Brunswick, NJ 08902"
934,"$1,350,000",5,4.5,"4,581Sq Ft","Partially Finished, Recreation Room, Storage ...",1,12,"$24,063",NaN,Single Family Residential,1988Year,7/100,25/100,"7 Golden Pond Dr,East Brunswick, NJ 08850"


First we identify the number of null values present in the data and the assigned data types

In [85]:
# df.drop(['Property Type'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 936 entries, 0 to 935
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Price           936 non-null    object
 1   Beds            936 non-null    object
 2   Baths           936 non-null    object
 3   Sq-ft           936 non-null    object
 4   Basement        936 non-null    int64 
 5   Garage          936 non-null    int64 
 6   Rooms           709 non-null    object
 7   Annual Tax      804 non-null    object
 8   Style           906 non-null    object
 9   Year Built      906 non-null    object
 10  Walkable Score  924 non-null    object
 11  Bikeable Score  924 non-null    object
 12  Address         936 non-null    object
 13  City            936 non-null    object
 14  Zip             933 non-null    object
dtypes: int64(2), object(13)
memory usage: 109.8+ KB


In [94]:
df

,Price,Beds,Baths,Sq-ft,Basement,Garage,Rooms,Annual Tax,Style,Year Built,Walkable Score,Bikeable Score,Address,City,Zip
0,665000,4,2.5,2418,1,1,NaN,11409,Single Family Residential,1979,44,40,"43 Sand Hills Rd,Kendall Park, NJ 08824",Kendall Park,08824
1,249900,2,1,936,0,0,4,5189,Condo/Co-op,1977,28,41,"280 College Dr #280,Edison, NJ 08817",Edison,08817
2,399000,3,1,1008,1,0,5,4926,Single Family Residential,1963,30,32,"616 Plainfield Ave,Piscataway, NJ 08854",Piscataway,08854
3,599900,3,3.5,NaN,1,1,6,4385,Single Family Residential,1954,19,32,"313 Wadsworth Ave,South Plainfield, NJ 07080",South Plainfield,07080
4,289000,2,1,NaN,1,0,5,4154,Single Family Residential,1923,41,35,"214 Seaview Ave,Old Bridge, NJ 08879",Old Bridge,08879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,459900,4,2,1468,1,1,9,4220,Single Family Residential,1926,81,67,"12 Holmes Ave,South River, NJ 08882",South River,08882
932,894990,4,2.5,2999,1,0,NaN,NaN,NaN,NaN,0,25,"Brookdale Plan,Old Bridge, NJ 08857",Old Bridge,08857
933,650000,4,3.5,2672,1,1,11,12497,Single Family Residential,1961,57,55,"541 Allgair Ave,North Brunswick, NJ 08902",North Brunswick,08902
934,1350000,5,4.5,4581,1,1,12,24063,Single Family Residential,1988,7,25,"7 Golden Pond Dr,East Brunswick, NJ 08850",East Brunswick,08850


In [95]:
# Removing the $, ',' from price and Annual Tax
# Removing the ',', Sq Ft from Sq-ft
# Filter out all the lots for sale
# Remove '/100' from walkable score, bikeable score
# change Price, Sq-ft, Annual Tax, walkable, bikeable to int
# change Beds, Baths to float
# change year built to datetime
# split address

# address_to_drop = df.iloc[0]['Address']
# df = df[df['Address'] != address_to_drop]

# df['Price'] = df['Price'].str.replace('[$,+]', '', regex=True)

# df['Sq-ft'] = df['Sq-ft'].str.replace(',', '', regex=False)
# df = df[(df['Sq-ft'].str.contains('Sq Ft', case=False) | (df['Sq-ft'] == '—')) & (~df['Sq-ft'].str.contains('lot', case=False))]
# df['Sq-ft'] = df['Sq-ft'].str.replace('Sq Ft', '', regex=False)
# df = df.replace('—', np.nan)

df['Rooms'] = df['Rooms'].str.replace(r'\D', '', regex=True)
df = df.replace('', np.nan)

df['Year Built'] = df['Year Built'].str.replace('Year','')
df['Year Built'] = df['Year Built'].str.replace('—','')

df['Basement'] = df['Basement'].str.replace('0', 'None')
df['Basement'] = df['Basement'].str.split(',').str[0]
df['Basement'] = df['Basement'].replace({'No': 0, 'nan': 0, ' None': 0, 'None': 0})
df['Basement'] = df['Basement'].apply(lambda x: 1 if x != 0 else 0)

df['Annual Tax'] = df['Annual Tax'].str.replace('[$,]', '', regex=True)
df['Walkable Score'] = df['Walkable Score'].str.replace('/100', '', regex=False)
df['Bikeable Score'] = df['Bikeable Score'].str.replace('/100', '', regex=False)

df['City'] = df['Address'].str.split(',').str[1]
df['Zip'] = df['Address'].str.split('NJ').str[1]

df[['Price', 'Beds', 'Baths', 'Sq-ft', 'Rooms', 'Annual Tax', 'Walkable Score', 'Bikeable Score']] = df[['Price', 'Beds', 'Baths', 'Sq-ft', 'Rooms', 'Annual Tax', 'Walkable Score', 'Bikeable Score']].astype(
{
    'Price': int, 'Beds': float, 'Baths': float, 'Sq-ft': float, 'Rooms': float,'Annual Tax': float,
    'Walkable Score': float, 'Bikeable Score': float
})

Will leave out Property Type and Style in our analysis due to too many values.
An improvement when collecting data should be to provide a list of options with other as an option for any that do not come under the given categories.

In [ ]:
df2 = df.drop(['Property Type', 'Style', 'Address'], axis=1)
df2['Avg Price/Sqft'] = round(df2['Price']/df2['Sq-ft'], 2)

In [ ]:
df2

In [ ]:
df2.isnull().sum()

In [ ]:
# Where beds is not mentioned and baths are those homes are usually a studio so
df2.loc[(df2['Beds'].isnull()) & (df2['Baths'].notnull()), 'Beds'] = 0

# substitute null value for rooms with beds + 3 (kitchen, family, dining)
df2.loc[df2['Rooms'].isnull(), 'Rooms'] = df2['Beds']+3

In [ ]:
df.to_csv('real_estate_cleaned.csv', index = False)